In [ ]:
from numpy import *
from matplotlib.pyplot import plot, show, title
from scipy.special import factorial

# 我们今天要干什么？
- 复习Python和Notebook基本操作
- 绘制双原子分子振动能级布居
- Python语法之二：列表(list)和循环
- 计算从振动基态到激发态的Franck-Condon因子
- 绘制一维势箱、三维势箱的DOS
- 利用休克尔模型观察能带的形成

# 1. 转动能级布居
转动光谱中，谱线高度先升高后降低，这是由于振动能级的布居先升高后降低导致的。

![rotation](./fig/rotation.png)

转动能级的布居公式为：
$$𝑛_𝐽 \propto (2𝐽+1)𝑒^{\frac{−ℎ𝑐𝐵 ̃𝐽(𝐽+1)}{𝑘𝑇}}$$
其中$J$是转动量子数，$h$是普朗克常数，$c$是光速，$B$是转动常数，$k$是玻尔兹曼常数，$T$是温度。指前因子是由能级简并引起的。

In [ ]:
from scipy.constants import h, c, k

取CO的转动常数，$B=1.93 \, \textrm{cm}^{-1} = 193 \, \textrm{m}^{-1} $

In [ ]:
T = 300
B = 193
J = arange(50)

In [ ]:
population = (2 * J + 1) * exp(- h * c * B * J * (J+1) / (k * T))
plot(J, population)

使用`vlines`函数画分立的竖线可以模拟光谱

In [ ]:
from matplotlib.pyplot import vlines
vlines(J, zeros_like(J), population)

## 试一试
氢分子的转动常数为$60.85 \, \textrm{cm}^{-1}$，HCl分子的转动常数为$10.59 \, \textrm{cm}^{-1}$，这两个分子的转动能级布居和CO相比有什么不同？为什么？

## 实验（写入实验报告）
绘制CO转动能级在温度$T$下的布居和光谱形状，其中$T$为你的学号除以999的余数+1。

# 2. Python语法之2：列表(`list`)和循环
将一堆Python的变量用`[]`括起来，就形成了一个列表(`list`)。注意到列表中可以包括各种不同的元素。

In [ ]:
[pi, 2, h, c, arange(5)]

利用`for xx in xx`的语法，可以**迭代**一个数组。为了显示迭代的效果，我们使用了`print`函数

In [ ]:
for i in arange(5):
    print(i)

利用`append`操作可以给列表添加元素

In [ ]:
l = [2, pi]
for i in arange(5):
    l.append(i)
l

当列表中都是数字时，可以用`plot`函数将`list`画出

In [ ]:
plot(l)

# 3. Franck-Condon因子
首先回忆谐振子的波函数：
$$\psi_(\xi) = N_v H_v(\xi) e^{-\frac{\xi^2}{2}}$$
其中$\xi = \frac{x}{\alpha}$, $\alpha = \left ( \frac{\hbar^2}{mk} \right ) ^{\frac{1}{4}} = \sqrt{\frac{\hbar}{m\omega}}$, $N_v$为归一化常数，$H_v(\xi)$为厄米多项式。 
即谐振子波函数为归一化因子、厄米多项式和一个高斯型分布的乘积。

上次实验中，已经绘制过谐振子的波函数

In [ ]:
from scipy.special import eval_hermite as hermite
limit = 10
xi = linspace(-limit, limit, 10000)
nu_1 = 0
wfn1 = hermite(nu_1, xi) * e ** (-xi ** 2/ 2)
plot(xi, wfn1)

## 0-0跃迁的Franck-Condon因子随平衡位置偏移距离的关系
Franck-Condon原理指出在电子跃迁的瞬间，核构型不变。电子跃迁时核的初态和末态振动波函数的重叠大小称为Franck-Condon因子，以下简称FC因子。

Franck-Condon因子越大电子跃迁越容易发生。

![fc1](./fig/fc1.png)

振动波函数重叠大小的求法是求积分：
$$
\textrm{FC} = \left ( \int \psi^*(\xi) \psi(\xi) dx \right)^2
$$
计算机当然可以求积分，不过本质上都是离散地求的，即将积分转换为求和：
$$
\textrm{FC} = \left ( \sum_\xi \psi^*(\xi) \psi(\xi) \right)^2
$$
现假设一个谐振子的平衡位置位于原点，其基态波函数即为上图，另一个波函数的平衡位置在$D$，其基态波函数为下图。
由于$\xi$实际上是经过归一化的距离，那么这里的$D$也是经过了归一化的

In [ ]:
D = 2
nu_2 = 0
wfn2 = hermite(nu_2, xi-D) * e ** (-(xi-D) ** 2 / 2)
plot(xi, wfn2)

我们可将两张图及其乘积叠在一起，可见其乘积只有在两个波函数都比较大时才比较大，否则接近于0

In [ ]:
plot(xi, wfn1)
plot(xi, wfn2)
plot(xi, wfn1 * wfn2)

接下来，只需对乘积求和即可得FC因子了。但是要注意这里我们的波函数都是未归一化的，如何归一化呢？

In [ ]:
((wfn1 * wfn2).sum() / sqrt((wfn1 ** 2).sum() * (wfn2 ** 2).sum())) ** 2

下面我们利用一个循环来计算不同$D$下的FC因子，并记录到一个`list`中，最后画出FC因子与$D$的关系。

In [ ]:
D_array = linspace(-10, 10, 100)
FC_list = []
for D in D_array:
    wfn2 = hermite(nu_2, xi-D) * e ** (-(xi-D) ** 2 / 2)
    FC = ((wfn1 * wfn2).sum() / sqrt((wfn1 ** 2).sum() * (wfn2 ** 2).sum())) ** 2
    FC_list.append(FC)
plot(D_array, FC_list)

从图中可见，当$D=0$时两个波函数的重叠最大，为1，这说明我的计算进行了正确的归一化。当$D$增大时，FC因子迅速降低到0。

下图中给出了PPT上的0-0跃迁Franck-Condon因子与平衡位置间距的关系，与我们画的图是一致的。平衡位置间距越远，0-0跃迁的强度越低，这和物理化学直觉也是符合的

![fc1](./fig/fc2.png)

## 一定平衡位置间距下0-M跃迁的FC因子与M的关系
0-M跃迁即从一个谐振子的基态到另一个谐振子的第M激发态的跃迁。
PPT上给出了当平衡间距为$q_0$时，0-M跃迁的FC因子为：
$$
\textrm{FC}(0, M) = \frac{S^M}{M!}e^{-S}
$$
其中S称为黄昆因子，$S=\frac{\omega D^2}{2}$，$\omega$是谐振子振动频率。
下面我们通过“暴力计算”的方式，对这个式子做一下验证。

In [ ]:
omega = 1
D = 2
S = omega * D ** 2 / 2
S

In [ ]:
M_array = arange(10)
FC_list = []
for nu_2 in M_array:
    wfn2 = hermite(nu_2, xi-D) * e ** (-(xi-D) ** 2 / 2)
    FC = ((wfn1 * wfn2).sum() / sqrt((wfn1 ** 2).sum() * (wfn2 ** 2).sum())) ** 2
    FC_list.append(FC)
plot(M_array, FC_list)
plot(S ** M_array / factorial(M_array) * exp(-S))

为什么只看到一条曲线呢？因为两条曲线完全重叠。
下面一张图通过设置`marker`的方式来区分两条曲线，注意我们可以通过`xx=xx`而不仅仅是位置来传递参数。

In [ ]:
plot(M_array, FC_list, marker="o")
plot(S ** M_array / factorial(M_array) * exp(-S), marker="x")

## 实验（写入实验报告）
通过改变$D$改变$S$，绘制FC因子-$M$曲线，并回答曲线的最高点和$S$的关系是什么？

# 4. 一维/三维势箱态密度
一维势箱的能级公式为：
$$
E_n = \frac{\hbar^2 \pi^2 n^2}{2mL^2}
$$
其中$m$为粒子质量，$L$为势箱长度。对于一个确定的 体系，将$\frac{\hbar^2 \pi^2}{2mL^2}$视作常数，则能量只和$n$有关

In [ ]:
E_array = arange(1000) ** 2

利用直方图可以方便地查看数据的分布情况，即能态的密度

In [ ]:
from matplotlib.pyplot import hist
hist(E_array, bins=100);

对于三维势箱，同理可得：

In [ ]:
E_array = arange(100).reshape(-1, 1, 1) ** 2 + arange(100).reshape(1, -1, 1) ** 2 + arange(100).reshape(1, 1, -1) ** 2
hist(E_array.flatten(), bins=100, range=(0, 10000));

其中设置`range`是为了截去能量较高，没有穷举完全的部分。PPT中固体的DOS如下图所示：

![dos](./fig/dos.png)

## 实验（写入实验报告）
绘制二维势箱态密度，指出满足什么规律。

# 5. 利用休克尔模型观察能带的形成
我们在上次实验中已经计算过环状休克尔模型的能量，本次实验我们计算长链休克尔模型能量。

In [ ]:
alpha = 0
beta = -1

下图中，每条竖线代表一个能级：

In [ ]:
for n in [4, 20, 50, 100, 200, 500]:
    H = diag(ones(n)) * alpha + diag(ones(n-1) * beta, 1) + diag(ones(n-1) * beta, -1)
    eigvals = linalg.eigvalsh(H)
    vlines(eigvals, zeros_like(eigvals), ones_like(eigvals))
    title(n)
    show()

可见随着参与休克尔体系的轨道数目增加，能级越来越密集，最后倾向于连续。能带的带顶为2，带底为-2，宽度为4。

下面我们换一种画法，将能级绘制为直方图：

In [ ]:
for n in [4, 20, 50, 100, 200, 500]:
    H = diag(ones(n)) * alpha + diag(ones(n-1) * beta, 1) + diag(ones(n-1) * beta, -1)
    eigvals = linalg.eigvalsh(H)
    hist(eigvals, bins=20)
    title(n)
    show()

从直方图中我们可以读出，基于休克尔近似形成的能带的态在带边密度较大，而在带中间密度较低。

## 实验（写入实验报告）
改变$\alpha$和$\beta$，观察休克尔模型能带的带底、带顶、带宽和态度度如何变化，并简单解释